In [4]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Import the datetime module from the datetime library.
from datetime import datetime

In [5]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [8]:
# Use the tuple() function to display the latitude and longitude combinations.
for coordinate in coordinates:
    print(coordinate[0], coordinate[1])

82.03577500512847 9.332151398643873
45.55005079521115 49.72839358185368
77.03537576716596 -78.45560509657987
-59.37375809674919 128.52109959667536
19.50315160974199 110.5551168977367
-63.8605726103133 143.89411486503712
-59.27976861051259 9.20826030937718
40.10133460800961 36.09580808302994
31.382841925788057 -13.187885145884678
10.729410361162309 -113.00752304387917
20.02115657595455 166.0834481792882
-60.631374235822456 -59.794380993097704
38.92071907980227 176.40625082587997
-40.7538457244025 -45.59133297921599
22.01093780388362 146.57087495069192
29.674931055877266 -17.985197276625144
81.42148460657049 -129.27599185996536
9.487725945629535 -20.052381452364557
53.77619479347368 -0.27246161889496534
-80.43634804002356 -19.880813365637465
23.46494822052206 -111.29605012212919
57.578421783767396 -33.69921639224199
-12.819214193187534 160.8317564811437
23.470835745297208 9.680898313108287
-57.937834392562266 76.34050523174159
-87.26783892251997 -67.91651491726995
-35.51217366435765 -12.

0.5060498680612397 178.9010522120928
-76.23623208203965 32.55816483659217
-58.22179012180351 -137.4390347818624
75.86597878037693 141.3048772883476
-22.30609695201754 77.92883270831993
-80.70456882221981 -93.68927161165243
72.6415869626953 -145.2197819317934
60.19566417317233 32.47877972834334
-33.264932068511385 -179.24629834363446
-53.893421560690825 -121.42650259864023
27.75531488890408 -52.54310900855144
-2.0027471992044497 -81.53329567267959
-14.96026131250926 150.31934749720307
48.22769370874619 178.53195054581607
80.35815643633214 -33.23311126661281
-31.74329041633478 114.77076084130863
-30.355158500715675 176.7429952891323
-74.89589169266097 14.411782205765377
75.05657278413719 -156.40756159129143
21.991097316976663 53.99849117476461
-42.0152562489798 -75.71559911532401
25.116413137125576 84.3556783247218
-75.86611460581423 120.69018240293883
-52.04922358928884 39.0477518112404
-20.781553084573147 -106.30618714747642
-7.279431524436035 123.24558194760101
-65.04096662957775 153.

40.66400691117269 -128.98160224026228
36.92822609584728 135.00589583955002
32.30297637473383 -28.198864125275747
-31.38136944091471 -48.37411574467248
-45.195958082346294 148.82267365952958
26.554322365652737 -99.53031170764918
31.492731918630724 174.92091731109696
80.11133807166973 37.7429506765352
18.662865203885147 -89.07356665033289
-26.729189876555807 93.19354713303437
62.33849950500675 -33.04597848860911
41.80418387747832 -55.92301958419044
-46.71749755658373 -118.90907463606408
77.52766287171423 40.063335119885835
71.91234196950751 25.465789998818764
27.45800232666798 103.61006357302438
48.43980996930762 -88.34362015553039
21.800895939315964 121.6483113024596
0.6206936535432135 -80.75098871813492
-81.14313698298888 -175.97561750388422
56.1910984881184 -177.24139914216642
-64.85522686916919 -104.02656191531332
-31.80493743849363 129.55869678730795
-50.07530883841406 61.05148305480563
37.0026827381635 32.53649549957444
2.106882172258153 -56.15605131052226
86.155202104767 13.065800

In [9]:
from citipy import citipy
for coordinate in coordinates:
     print(citipy.nearest_city(coordinate[0], coordinate[1]).city_name,
          citipy.nearest_city(coordinate[0], coordinate[1]).country_code)

barentsburg sj
fort-shevchenko kz
qaanaaq gl
portland au
chengmai cn
hobart au
hermanus za
turhal tr
teguise es
san patricio mx
butaritari ki
ushuaia ar
nikolskoye ru
chuy uy
katsuura jp
santa cruz de la palma es
tuktoyaktuk ca
bubaque gw
kingston upon hull gb
ushuaia ar
todos santos mx
tasiilaq gl
kirakira sb
gat ly
souillac mu
ushuaia ar
jamestown sh
amderma ru
raudeberg no
rikitea pf
torbay ca
pasighat in
hengshui cn
arraial do cabo br
port alfred za
padang id
saleaula ws
mocambique mz
belushya guba ru
mayachnyy ru
rikitea pf
atuona pf
punta arenas cl
barry gb
orsha by
mataura pf
mataura pf
atuona pf
busselton au
ziro in
tommot ru
busselton au
arica cl
killybegs ie
pangnirtung ca
ancud cl
bambous virieux mu
tumannyy ru
alotau pg
mount gambier au
hofn is
taolanaro mg
tiksi ru
hermanus za
jamestown sh
grand gaube mu
punta arenas cl
barentsburg sj
mataura pf
atuona pf
rikitea pf
bredasdorp za
acarau br
the valley ai
marsh harbour bs
semirom ir
aklavik ca
rikitea pf
antofagasta cl
mar d

butaritari ki
tuktoyaktuk ca
bredasdorp za
cape town za
aflu dz
norman wells ca
geraldton au
sile tr
guerrero negro mx
bonavista ca
busselton au
qaanaaq gl
grand river south east mu
pundaguitan ph
busselton au
labutta mm
bluff nz
matara lk
toktogul kg
jamestown sh
atherton au
mar del plata ar
butaritari ki
bluff nz
punta arenas cl
tabou ci
east london za
sangre grande tt
albany au
karaul ru
bokoro td
hornepayne ca
mataura pf
port elizabeth za
provideniya ru
punta arenas cl
bengkulu id
davila ph
qaanaaq gl
gwanda zw
opunake nz
joshimath in
busselton au
taolanaro mg
coquimbo cl
tuatapere nz
port alfred za
pisco pe
nivala fi
cape town za
rungata ki
port elizabeth za
rikitea pf
deputatskiy ru
grand river south east mu
punta arenas cl
fairbanks us
syasstroy ru
vaini to
rikitea pf
saint george bm
salinas ec
samarai pg
nikolskoye ru
illoqqortoormiut gl
perth au
russell nz
bredasdorp za
barrow us
abu dhabi ae
ancud cl
nasriganj in
albany au
east london za
puerto ayora ec
maumere id
hobart au
k

tumannyy ru
nicolas bravo mx
carnarvon au
tasiilaq gl
saint-pierre pm
rikitea pf
ostrovnoy ru
honningsvag no
zhaotong cn
thunder bay ca
taitung tw
muisne ec
vaini to
provideniya ru
punta arenas cl
flinders au
taolanaro mg
cumra tr
grand-santi gf
barentsburg sj
mataura pf
plainview us
burnie au
richards bay za
bambous virieux mu
ilulissat gl
busselton au
ushuaia ar
sao filipe cv
yulara au
kifri iq
queenstown za
faanui pf
sentyabrskiy ru
ushuaia ar
puerto ayora ec
busselton au
ushuaia ar
luderitz na
taolanaro mg
bethel us
punta arenas cl
sao joao da barra br
botwood ca
ponta do sol cv
carnarvon au
nikolskoye ru
yellowknife ca
manggar id
urusha ru
santa clara mx
butaritari ki
albany au
belmonte br
east london za
uni ru
salme ee
east london za
torbay ca
acapulco mx
tekeli kz
castro cl
victoria sc
new norfolk au
saint george bm
port alfred za
lorengau pg
juneau us
provideniya ru
ashoknagar in
bredasdorp za
cockburn town bs
soledade br
atuona pf
hilo us
upernavik gl
mys shmidta ru
ostrovnoy 

In [25]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)
#print(cities)

613

In [11]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

Beginning Data Retrieval     
-----------------------------


In [13]:
# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

In [14]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=14de9ee128af72181dfcef1721e8ad4d


In [16]:
# Loop through all the cities in our list.
for i in range(len(cities)):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + cities[i]

In [19]:
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Processing Record 14 of Set 37 | barentsburg
City not found. Skipping...
Processing Record 15 of Set 37 | fort-shevchenko
Processing Record 16 of Set 37 | qaanaaq
Processing Record 17 of Set 37 | portland
Processing Record 18 of Set 37 | chengmai
City not found. Skipping...
Processing Record 19 of Set 37 | hobart
Processing Record 20 of Set 37 | hermanus
Processing Record 21 of Set 37 | turhal
Processing Record 22 of Set 37 | teguise
Processing Record 23 of Set 37 | san patricio
Processing Record 24 of Set 37 | butaritari
Processing Record 25 of Set 37 | ushuaia
Processing Record 26 of Set 37 | nikolskoye
Processing Record 27 of Set 37 | chuy
Processing Record 28 of Set 37 | katsuura
Processing Record 29 of Set 37 | santa cruz de la palma
Processing Record 30 of Set 37 | tuktoyaktuk
Processing Record 31 of Set 37 | bubaque
Processing Record 32 of Set 37 | kingston upon hull
Processing Record 33 of Set 37 | todos santos
Processing Record 34 of Set 37 | tasiilaq
Processing Record 35 of S

Processing Record 34 of Set 40 | moose factory
Processing Record 35 of Set 40 | sioux lookout
Processing Record 36 of Set 40 | cherskiy
Processing Record 37 of Set 40 | srandakan
Processing Record 38 of Set 40 | utica
Processing Record 39 of Set 40 | atar
Processing Record 40 of Set 40 | vila franca do campo
Processing Record 41 of Set 40 | vaini
Processing Record 42 of Set 40 | provideniya
Processing Record 43 of Set 40 | ossora
Processing Record 44 of Set 40 | sitka
Processing Record 45 of Set 40 | lasa
Processing Record 46 of Set 40 | uthal
Processing Record 47 of Set 40 | kelowna
Processing Record 48 of Set 40 | barbar
City not found. Skipping...
Processing Record 49 of Set 40 | yavaros
Processing Record 50 of Set 40 | new norfolk
Processing Record 1 of Set 41 | olavarria
Processing Record 2 of Set 41 | jawhar
Processing Record 3 of Set 41 | zharkent
Processing Record 4 of Set 41 | kodiak
Processing Record 5 of Set 41 | bathsheba
Processing Record 6 of Set 41 | pevek
Processing Rec

Processing Record 21 of Set 44 | kavaratti
Processing Record 22 of Set 44 | villa carlos paz
Processing Record 23 of Set 44 | novyye burasy
Processing Record 24 of Set 44 | karamay
City not found. Skipping...
Processing Record 25 of Set 44 | wanning
Processing Record 26 of Set 44 | cayenne
Processing Record 27 of Set 44 | butare
Processing Record 28 of Set 44 | boden
Processing Record 29 of Set 44 | tiznit
Processing Record 30 of Set 44 | sisimiut
Processing Record 31 of Set 44 | millinocket
Processing Record 32 of Set 44 | alta floresta
Processing Record 33 of Set 44 | meulaboh
Processing Record 34 of Set 44 | rognan
Processing Record 35 of Set 44 | lolua
City not found. Skipping...
Processing Record 36 of Set 44 | krasnoselkup
Processing Record 37 of Set 44 | shaoyang
Processing Record 38 of Set 44 | roma
Processing Record 39 of Set 44 | capaccio
Processing Record 40 of Set 44 | macamic
Processing Record 41 of Set 44 | oruro
Processing Record 42 of Set 44 | la rioja
Processing Record

Processing Record 6 of Set 48 | vanimo
Processing Record 7 of Set 48 | vryburg
Processing Record 8 of Set 48 | saint anthony
Processing Record 9 of Set 48 | weligama
Processing Record 10 of Set 48 | abalak
Processing Record 11 of Set 48 | erzin
Processing Record 12 of Set 48 | fort nelson
Processing Record 13 of Set 48 | ixtepec
Processing Record 14 of Set 48 | coihaique
Processing Record 15 of Set 48 | alyangula
Processing Record 16 of Set 48 | mana
Processing Record 17 of Set 48 | ypsilanti
Processing Record 18 of Set 48 | puerto maldonado
Processing Record 19 of Set 48 | hermiston
Processing Record 20 of Set 48 | ishigaki
Processing Record 21 of Set 48 | formiga
Processing Record 22 of Set 48 | eureka
Processing Record 23 of Set 48 | mikuni
Processing Record 24 of Set 48 | ponta delgada
Processing Record 25 of Set 48 | guerrero
Processing Record 26 of Set 48 | nicolas bravo
Processing Record 27 of Set 48 | honningsvag
Processing Record 28 of Set 48 | zhaotong
Processing Record 29 of

In [26]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(cities)
city_data_df.head(10)

,0
0,barentsburg
1,fort-shevchenko
2,qaanaaq
3,portland
4,chengmai
5,hobart
6,hermanus
7,turhal
8,teguise
9,san patricio


In [27]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")

FileNotFoundError: [Errno 2] No such file or directory: 'weather_data/cities.csv'